# 题目

> 给你一个整数数组 nums 和一个整数 k ，请你返回其中出现频率前 k 高的元素。  
你可以按任意顺序返回答案。

# 方法一：堆

> 基于比较地将频率排序并返回第k高的元素的时间复杂度为 $O(nlogn)$ ，使用堆进行排序可以将时间复杂度降为 $O(nlogk)$ 。  
堆是用列表形式表现的完全二叉树，具有以下性质，方便使用位操作：  
1. heap[0]（堆使用一个默认值0进行占位），堆的根节点为heap[1]；
2. 对于堆中的任意一个索引为i的父节点，其左子节点的索引为2i（位操作：i << 1），其右子节点的索引为2i+1（位操作：i << 1 | 1）；
3. 对于堆中任意一个索引为i的子节点，其父节点（若有）的索引为i//2（位操作：i >> 1）；
4. 最小堆（小顶堆）的所有父节点都小于子节点。  

> 堆中存在两个重要操作：  
1. 上浮 sift up: 向堆尾新加入一个元素，堆规模 +1，依次向上与父节点比较，如小于父节点就交换;
2. 下沉 sift down: 从堆顶取出一个元素（堆规模 -1，用于堆排序）或者更新堆中一个元素（本题），依次向下与子节点比较，如大于子节点就交换。  

> 因此，为了求频率最大的k个元素，构建一个包含k+1个元素（包含占位的0）的最小堆并维护，操作如下：  
1. 得到一个列表，其中包含数组中所有数的频率；
2. 使用数组中前k个数构造一个堆；
3. 遍历规模k之外的数据，大于堆顶则入堆，下沉维护规模为k的最小堆 minheap。
4. 输出这个堆。

## 复杂度

- 时间复杂度: $O(nlogk)$ ，其中 $n$ 是数组长度， $k$ 是要求的排序值。

> 统计数组中各个数出现的频率的时间复杂度为 $O(n)$ ，对n个数进行堆（大小为k+1的堆）操作的时间复杂度为 $O(logk)$ ，因此时间复杂度为 $O(nlogk)$ 。

- 空间复杂度: $O(nlogk)$ ，其中 $n$ 是数组长度， $k$ 是要求的排序值。

> 统计频率的哈希表大小为 $O(n)$ ，而堆的大小为 $O(k)$ ，共计为 $O(n)$ 。

## 代码

In [1]:
import collections

In [2]:
def topKFrequent(nums, k):
    def sift_down(arr, root, k):
        """下沉log(k),如果新的根节点>子节点就一直下沉"""
        val = arr[root]  # 从堆顶取出新加入节点的值（一个元组，包括：1、数组中的数；2、其出现的频率）
        while root << 1 < k:  # 若新节点的左子节点小于k（不超过堆大小）
            child = root << 1  # 取出新节点的左子节点
            # 选取左右孩子中小的与父节点交换（保证交换完成后父节点始终小于两个子节点）
            if child|1 < k and arr[child|1][1] < arr[child][1]:  # 如果新节点的右子节点（child|1）不超过范围（<k），且值比左子节点更小
                child |= 1  # 则取出新节点的右子节点
            if arr[child][1] < val[1]:  # 若新节点的值大于较小子节点的值，则交换
                arr[root] = arr[child]
                root = child
            else:  # 否则，说明其已经有序排列，退出
                break
        arr[root] = val  # 给来到正确位置的新节点赋值

    def sift_up(arr, child):
        """上浮log(k),如果新加入的节点<父节点就一直上浮"""
        val = arr[child]  # 取出新加入子节点的值（一个元组，包括：1、数组中的数；2、其出现的频率）
        # 若其父节点（child >> 1表示父节点）存在（即该子节点不是根节点），且子节点值小于父节点值
        while child >> 1 > 0 and val[1] < arr[child >> 1][1]:
            arr[child] = arr[child>>1]  # 则将父节点值赋给子节点
            child >>= 1  # 将子节点位置指向父节点的位置
        # 最终，给来到正确位置的子节点赋值
        arr[child] = val

    stat = collections.Counter(nums)  # 统计数组中所有数出现的频率
    stat = list(stat.items())  # 返回一个列表，其元素是元组，包括：1、数组中的数；2、其出现的频率
    heap = [(0,0)]  # 构建堆（起始包括(0,0)，用于占位），其中每个元素（元组）包括：1、数组中的数；2、其出现的频率

    # 构建规模为k+1的堆,新元素加入堆尾,上浮
    for i in range(k):
        heap.append(stat[i])
        sift_up(heap, len(heap)-1)  # len(heap)-1就是新加入元素的位置
    
    # 维护规模为k+1的堆,如果新元素大于堆顶,入堆,并下沉
    for i in range(k, len(stat)):  # 从k到stat末尾遍历
        if stat[i][1] > heap[1][1]:  # 如果当前元素值大于根节点值
            heap[1] = stat[i]  # 将当前元素放入对顶
            sift_down(heap, 1, k+1)  # 并下沉
    
    # 返回构建好的堆
    return [item[0] for item in heap[1:]]

#### 测试一

In [5]:
nums = [1,1,1,2,2,3]
k = 2
topKFrequent(nums, k)

[2, 1]

#### 测试二

In [6]:
nums = [1]
k = 1
topKFrequent(nums, k)

[1]